In [47]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from datetime import datetime
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# load iris
iris = load_iris()
X = iris["data"][:,[2,3]]
y = (iris["target"] == 0).astype(np.int)

# train perceptron
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)
print(accuracy_score(y, per_clf.predict(X)))
"""
# load mnist
scaler = StandardScaler()
df_train = pd.read_csv("../datasets/mnist_train.csv")
X_train = scaler.fit_transform(df_train.iloc[:].drop("5", axis=1).values)
y_train = df_train.iloc[:]["5"].values.reshape(-1,1)
df_test = pd.read_csv("../datasets/mnist_test.csv")
X_test = scaler.transform(df_test.drop("7", axis=1).values)
y_test = df_test["7"].values.reshape(-1,1)

# train MLP with tensorflow (high-level API)
start_time = time.time()
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], 
                                         n_classes=10, feature_columns=feature_cols)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)
y_pred = dnn_clf.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))
print(accuracy_score(y_test, y_pred["classes"])) # 0.9795 within 122.53914904594421 seconds

# train MLP with sklearn
start_time = time.time()
dnn_clf = MLPClassifier(hidden_layer_sizes=[300,100], batch_size=50, max_iter=40000)
dnn_clf.fit(X_train, y_train)
y_pred = dnn_clf.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))
print(accuracy_score(y_test, y_pred)) # 0.9738 within 86.09499192237854 seconds
"""
# train MLP with tensorflow (low-level API)
"""
with tf.variable_scope("param", reuse=tf.AUTO_REUSE):
    n_inputs = tf.get_variable("n_inputs", shape=(), 
                                 initializer=tf.constant_initializer(28*28))
    n_hidden1 = tf.get_variable("n_hidden1", shape=(), 
                               initializer=tf.constant_initializer(300))
    n_hidden2 = tf.get_variable("n_hidden2", shape=(), 
                               initializer=tf.constant_initializer(100))
    n_outputs = tf.get_variable("n_outputs", shape=(), 
                               initializer=tf.constant_initializer(10))
"""
tf.reset_default_graph()
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        #with tf.variable_scope("param", reuse=True):
        n_inputs = int(X.get_shape()[1])
        stddev = 2. / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
with tf.name_scope("dnn"):
    """
    with tf.variable_scope("param", reuse=True):
        n_hidden1 = tf.get_variable("n_hidden1")
        n_hidden2 = tf.get_variable("n_hidden2")
        n_outputs = tf.get_variable("n_outputs")
    """
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=tf.cast(tf.reshape(y,[-1]), tf.int32), logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
file_writer = tf.summary.FileWriter(logdir, graph=tf.get_default_graph())
file_writer.flush()
        

1.0


/usr/local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
